## 10. 얼굴 인식(Hear Cascade)

In [33]:
import cv2 as cv

CHA = "../images/cha.jpg"
SEO = "../images/seo.jpg"
HARRY = "../images/harry.jpg"
HER = "../images/her.jpg"
EYE = "../images/eye.jpg"

FACE_CASCADE = "../cascade/haarcascade_frontalface_default.xml"
EYE_CASCADE = "../cascade/haarcascade_eye.xml"

In [29]:
# 얼굴 인식
face_cascade = cv.CascadeClassifier(FACE_CASCADE)

img = cv.imread(SEO)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=20, minSize=(10,10)
)

if len(faces):
    for face in faces:
        x, y, width, height = face
        cv.rectangle(img, (x,y), (x+width, y+height), (0,255,0), 2, cv.LINE_AA)

cv.imshow("img", img)
cv.waitKey(0)
cv.destroyAllWindows()


In [39]:
# 눈 인식
eye_cascade = cv.CascadeClassifier(EYE_CASCADE)

img = cv.imread(CHA)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

eyes = eye_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=2, maxSize=(20,20)
)

if len(eyes):
    for eye in eyes:
        x, y, width, height = eye
        cv.rectangle(img, (x,y), (x+width, y+height), (255,0,0), 2, cv.LINE_AA)

cv.imshow("img", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [70]:
# 실습. 캠화면에 적용
face_cascade = cv.CascadeClassifier(FACE_CASCADE)
eye_cascade = cv.CascadeClassifier(EYE_CASCADE)

cap = cv.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=20, minSize=(10,10)
    )
    
    if len(faces):
        for face in faces:
            x, y, width, height = face
            cv.rectangle(frame, (x,y), (x+width, y+height), (0,0,255), 2, cv.LINE_AA)
            cv.putText(frame, "Face", (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    
    eyes = eye_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=10, maxSize=(30,30)
    )
    
    if len(eyes):
        for eye in eyes:
            x, y, width, height = eye
            cv.rectangle(frame, (x,y), (x+width, y+height), (0,255,0), 2, cv.LINE_AA)
            cv.putText(frame, "Eye", (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
    
    cv.imshow("Camera", frame)
    
    if cv.waitKey(1) == ord("q"):
        break

cap.release()
cv.destroyAllWindows()

In [67]:
# 실습. 귀여운 눈 덮어 씌우기
eye_cascade = cv.CascadeClassifier(EYE_CASCADE)

img = cv.imread(HER)
eyes_img = cv.imread(EYE)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

eyes = eye_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=5, maxSize=(100,100)
)


if len(eyes):
    for i, eye in enumerate(eyes):
        x, y, width, height = eye
        resized_eye = cv.resize(eyes_img, (width, height))
        
        img[y:y+height, x:x+width] = resized_eye


cv.imshow("IMG", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [69]:
# 실습. 귀여운 눈 덮어 씌우기(영상 ver)

eye_cascade = cv.CascadeClassifier(EYE_CASCADE)
video = cv.VideoCapture("../videos/move.mp4")
eyes_img = cv.imread(EYE)

while True:
    ret, frame = video.read()
    
    if not ret:
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    eyes = eye_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=100, maxSize=(1000,1000)
    )
    
    if len(eyes):
        for eye in eyes:
            x, y, width, height = eye
            resized_eye = cv.resize(eyes_img, (width, height))
            frame[y:y+height, x:x+width] = resized_eye
    
    
    resized_frame = cv.resize(frame, None, fx=0.25, fy=0.25)
    cv.imshow("Video", resized_frame)
    
    if cv.waitKey(1) == ord("q"):
        break

video.release()
cv.destroyAllWindows()

In [71]:
# 실습. 웹캠을 통해 들어오는 영상을 이용해 움직임이 감지될 경우 해당 프레임을 캡쳐해서 파일로 저장하는 프로그램 만들기

cap = cv.VideoCapture(0)
backSub = cv.createBackgroundSubtractorMOG2()
count = 1
capture_cooldown = 0

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    Sub = backSub.apply(frame)
    
    contours, _ = cv.findContours(Sub, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    motion_detected = False
    for contour in contours:
        area = cv.contourArea(contour)
        if area > 5000:
            motion_detected = True
            break
    
    if motion_detected and capture_cooldown == 0:
        filename = f"../output/capture_{count}.png"
        cv.imwrite(filename, frame)
        print(f"{filename.split('/')[2]} 저장!")
        count += 1
        capture_cooldown = 30
    
    if capture_cooldown > 0:
        capture_cooldown -= 1
    
    cv.imshow("Camera", frame)
    
    key = cv.waitKey(1)
    
    if key == ord("c"):
        filename = f"../output/capture_{count}.png"
        cv.imwrite(filename, frame)
        print(f"{filename.split('/')[2]} 저장!")
        count += 1
    
    elif key == ord("q"):
        break

cap.release()
cv.destroyAllWindows()

capture_1.png 저장!
capture_2.png 저장!
capture_3.png 저장!
capture_4.png 저장!
capture_5.png 저장!
capture_6.png 저장!
